# Text Data

Trying to construct models that understand text falls under the field of *natural language processing*. This is a field of enormous practical importance: chatbot, automated translation and generated new articles area few notable applications. In this notebook we will look into some basic ways of processing text data.

Below is what you might get in a typical dataset of review data:

In [ ]:
#Text data
corpus = [
    "This is good.",
    "This is bad.",
    "This is very good.",
    "This is not good.",
    "This is not bad.",
    "This is...is bad."
]

ratings = [
    1,
    0,
    1,
    0,
    1,
    0
]

When analyzing review data the typical goal is to predict a single value, the rating, from the written text. In the case of chatbot and automated translation, where one single value is not sufficient to represent the meaning of text, a vector is outputed by the model instead.

### A. N-gram

Let us count the number of times each word appears in a sample. This is called *unigram* in natural language processing. To do so, we will use ```CountVectorizer``` of scikit-learn:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


Use ```get_feature_names()``` to see which word each column represents:

The word-count vector can now be used with a suitable model to conduct language processing. Here we will simply use a logit model:

In [ ]:
#Logistic regression
from sklearn.linear_model import LogisticRegression


Which phrases do our model have a difficulty understanding? Why might that be the case?

Now let us take a look at the estimated coefficients:

Take a look at the coefficients of each word. Can you see what is wrong with our model? One thing you might notice is that 'is' has a very negative coefficient while 'very' has very a positive coefficient, even though these words do not have such connotations themselves.  

When we start counting combination of words instead of individual words, what we have is *n-gram*. ```CountVectorizer``` allows us to specify the range of words we wish to consider via the option ```ngram_range```:

Now let us try running the logistic regression again:

Much better!

### B. IMDB Movie Review

Now let us try something real. We will analyse a sample of <a href="https://www.imdb.com/">IMDB</a> movie reviews, trying to predict the rating a user gives based on his written review. We will be using a pre-processed version of the data, but the original text data can be found <a href="http://ai.stanford.edu/~amaas/data/sentiment/">here</a>.

First let us import the data:

In [ ]:
import numpy as np
imdb = np.load("../Data/imdb.npz", allow_pickle=True) 

Now take a look what is inside this file:

In [ ]:
for d in imdb:
    print(d)

How many samples do we have?

In [ ]:
print(imdb["x_train"].shape)
print(imdb["x_test"].shape)

What is inside each X sample?

In [ ]:
imdb["x_train"][0]

Words are encoded by their frequency-of-apperance ranking in the data. This allows us to easily delete words that either
- appear frequently but add little to the meaning of the text (e.g. articles, conjunctions and prepositions), or
- appear too infrequently to be of use.

What is inside each y sample?

In [ ]:
imdb["y_train"][0]

We will now repeat what we have done previously:

In [ ]:
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize

#Preprocess data

How well does our model do?

In [ ]:
model = LogisticRegression()
model.fit(x_train,y_train)
print(model.score(x_train,y_train))
print(model.score(x_test,y_test))

### C. Lemmatization

Consider the following corpus of text, modified from the original one:

In [ ]:
# Text data
corpus2 = [
    "Apple is good.",
    "Apple was bad.",
    "Apples are good.",
    "Apples were not good.",
    "Apple is not bad.",
    "Apples were...are bad."
]

Having plurals complicates our analysis: `CountVectorizer` will treat 'Apple' and 'Apples' as two distinct words, unncessarily splitting the samples for apples. Similarly, 'is' and 'are' are both forms of the verb 'to be', so they should be considered as one word. What we need is *lemmatization*, which is the process of grouping together the inflected forms of a word for use in analysis.

We will be using <a href="https://textblob.readthedocs.io/en/dev/index.html">TextBlob</a>, a library for processing textual data. TextBlob in turn relies on <a href="http://www.nltk.org/">NLTK</a> (short for *Natural Language ToolKit*) to do some of the heavy lifting. Since NLTK does not come with all packages installed, we will need to first download the ones we need:

In [ ]:
import nltk
nltk.download('punkt') 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

The process goes as follows:
1. First convert each string to a `TextBlob` object. 
2. Split each string into sentences with the `.sentences` property if needed.
3. Split each string (or sentence) into words with the `.words` property.
4. Lemmatize each word with the `lemmatize()` method. 

Note that `lemmatize()` expects words to be in lowercase.

In [ ]:
# Use TextBlob to lemmatize the corpus
from textblob import TextBlob



The code above successfully grouped 'apples' with 'apple', but it failed to group 'is' and 'are'. The second sample gives us some hint as to what went wrong---'was' was somehow converted to 'wa'. What happened was that `lemmatize()` by default treats all words as nouns. To ensure proper conversion, we will need to provide it with each word's part of speech (POS).

First, we generate part-of-speech tags by using the `.tags` property of the `TextBlob` object:


In [ ]:
# Extract Penn Treebank POS


We can then providing `lemmatize()` with part-of-speech tags. Unfortunately it is not as simple as passing the POS tags from above. The reason is that NLTK generates tags base on the <a href="https://catalog.ldc.upenn.edu/LDC99T42">Penn Treebank</a> corpus, which uses different <a href="https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html">POS</a> tags than the <a href="https://wordnet.princeton.edu/documentation/wndb5wn">Wordnet</a> corpus that `lemmatize()` is based on. 

We therefore need to map the two POS systems before lemmatization:

In [ ]:
# Function to map Penn Treebank POS to Wordnet POS

# Convert Penn Treebank POS to Wordnet POS

# Lemmatize with POS


TextBlob and NLTK have many other useful features such as spelling correction and translation that you can explore on your own. One particularly useful feature is pre-trained sentiment analysis:

In [ ]:
# Sentiment analysis with TextBlob


### D. Chinese Text

One major issue with Chinese text is that there is no space between words. Unsurprisingly then, this is a major focus for Chinese natural language processing research.

They are multiple libraries for Chinese NLP. Here we will try out `jieba` and `pkuseg`.

In [5]:
text = '我愛吃北京餃子。'

# jieba default


# jieba paddle mode


# pkuseg


['我愛吃', '北京', '餃子', '。']
['我', '愛', '吃', '北京', '餃', '子', '。']
['我', '愛', '吃', '北京', '餃子', '。']


Things are much easier once we have the individual words. For example, we could immediately use ngram on the text.

We can also fetch POS:

In [ ]:
# jieba


# pkuseg


### E. Neural Network

Below is a simple LSTM neural network model that runs sentiment analysis on the IMDB data:

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train,y_train,x_test,y_test = resample(x_train,y_train,x_test,y_test,
                                         n_samples=1000)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(128))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

You should notice that training a neural network is several orders of magnitude slower than a n-gram model. Furthermore, the neural network model above is not more accurate than our simple n-gram model. One reason is that with so many parameters, neural network models need more than a thousand sample to achieve good results. You can try running the same script with more data on a computer with GPU and see whether you get better results.

### Further Readings
- <a href="https://github.com/dipanjanS/text-analytics-with-python">Text Analytics with Python</a> (or the <a href="https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72">free tutorial</a> by the same author on Towards Data Science.)